In [61]:
import numpy as np
import pandas as pd
from astropy.io import fits
from matplotlib import pyplot as plt

In [6]:
data = pd.read_csv('Data/ztf_crm/ztf_crm.csv', header=None)
data.head()

,0
0,lightcurve_ztf_20221020148032_000686_zg_c01_x_...
1,lightcurve_ztf_20221020148032_000686_zg_c01_x_...
2,lightcurve_ztf_20221020148032_000686_zg_c01_x_...
3,lightcurve_ztf_20221020148032_000686_zg_c01_x_...
4,lightcurve_ztf_20221020148032_000686_zg_c01_x_...


In [7]:
data.drop(data.index[-1], inplace=True)
data.tail()

,0
18997,lightcurve_ztf_20221020148032_000686_zg_c13_x_...
18998,lightcurve_ztf_20221020148032_000686_zg_c13_x_...
18999,lightcurve_ztf_20221020148032_000686_zg_c13_x_...
19000,lightcurve_ztf_20221020148032_000686_zg_c13_x_...
19001,lightcurve_ztf_20221020148032_000686_zg_c13_x_...


In [8]:
temp = np.array([np.array(name.split('_')[2:6]) for name in data[0].values])

data['rundate'] = temp[:,0]
data['runnumber'] = temp[:,1]
data['filter'] = temp[:,2]
data['chip'] = temp[:,3]
data

,0,rundate,runnumber,filter,chip
0,lightcurve_ztf_20221020148032_000686_zg_c01_x_...,20221020148032,000686,zg,c01
1,lightcurve_ztf_20221020148032_000686_zg_c01_x_...,20221020148032,000686,zg,c01
2,lightcurve_ztf_20221020148032_000686_zg_c01_x_...,20221020148032,000686,zg,c01
3,lightcurve_ztf_20221020148032_000686_zg_c01_x_...,20221020148032,000686,zg,c01
4,lightcurve_ztf_20221020148032_000686_zg_c01_x_...,20221020148032,000686,zg,c01
...,...,...,...,...,...
18997,lightcurve_ztf_20221020148032_000686_zg_c13_x_...,20221020148032,000686,zg,c13
18998,lightcurve_ztf_20221020148032_000686_zg_c13_x_...,20221020148032,000686,zg,c13
18999,lightcurve_ztf_20221020148032_000686_zg_c13_x_...,20221020148032,000686,zg,c13
19000,lightcurve_ztf_20221020148032_000686_zg_c13_x_...,20221020148032,000686,zg,c13


In [9]:
data.groupby(['rundate', 'runnumber', 'filter', 'chip', ]).count()

0
rundate        runnumber filter chip      
20221020148032 000686    zg     c01   1650
                                c02   1560
                                c03   1890
                                c04   1980
                                c05    810
                                c06   1290
                                c07   1380
                                c08   1290
                                c09   1230
                                c10   1260
                                c11   1890
                                c12   1590
                                c13   1182

In [ ]:
arr = np.zeros(data[data['chip'] == 'c05'].values.shape[0])

for i, name in enumerate(data[0][data['chip'] == 'c05'].values):
    source = pd.read_csv(f'Data/ztf_crm/{name}')
    flux = source['flux_hap_2'].values
    middle_flux_mean = flux[int(len(flux) / 2 - 50) : int(len(flux) / 2 + 50)].mean()
    arr[i] = middle_flux_mean

In [50]:
testnames = data[0][data['chip'] == 'c05'].values[np.argwhere(arr >= np.sort(arr)[-10]).flatten()] #get names of the 10 sources with highest mean flux

In [59]:
testlcs = []

for i, name in enumerate(testnames):
    source = pd.read_csv(f'Data/ztf_crm/{name}')
    testlcs.append(source['flux_hap_2'].values)


testlcs = np.array(testlcs)
testlcs

array([[3305.02783203, 3266.13623047, 3314.77246094, ..., 3031.54003906,
        3062.37890625, 3086.37451172],
       [3330.06689453, 3341.90673828, 3325.25439453, ..., 3091.9296875 ,
        3078.09130859, 3191.15551758],
       [3275.91381836, 3273.26831055, 3304.33862305, ..., 3168.70068359,
        3129.21557617, 3222.68969727],
       ...,
       [3247.24682617, 3274.52246094, 3241.46875   , ..., 3160.04418945,
        3202.9140625 , 3257.21972656],
       [3218.24487305, 3162.45092773, 3206.38037109, ..., 3210.76196289,
        3165.96630859, 3064.39550781],
       [3391.56201172, 3331.96704102, 3269.97998047, ..., 3196.30151367,
        3302.1328125 , 3313.92626953]], shape=(10, 3080))

In [65]:
np.save('Data/ztf_crm/testlcs_c05.npy', testlcs)